# Example Firefly usage

This builds off of the [minimal example](https://firefly.rcs.northwestern.edu/docs/data_reader/minimal_example.html) from our docs and the new [data selection example](https://github.com/ageller/Firefly/blob/main/src/firefly/ntbks/selecting_data.ipynb) from our GitHub repo.


As an example, we will generate two random particle sets (a sphere and a cube) both with a randomized attribute field.  We will load these data into Firefly.

This notebook also demonstrates how to retrieve data that you selected by hand in the Firefly app and then use those data with your notebook. 

In [1]:
import numpy as np

In [2]:
# Load your data. Here we're creating random data.

# one particle set (sphere)
coords_sphere = np.random.randn( 20000, 3 )
fields_sphere = np.random.random(size=coords_sphere[:,0].size)

# a second particle set (cube)
coords_cube = np.random.rand( 20000, 3 ) - 0.5
fields_cube = np.random.random(size=coords_cube[:,0].size)

## Format Data for Firefly

Here are the docs for [Reader](https://firefly.rcs.northwestern.edu/docs/reference/api/classes/firefly.data_reader.Reader.html#firefly.data_reader.Reader) and [ParticleGroup](https://firefly.rcs.northwestern.edu/docs/reference/api/classes/firefly.data_reader.ParticleGroup.html#firefly.data_reader.ParticleGroup) and also [a tutorial for using these](https://firefly.rcs.northwestern.edu/docs/data_reader/reader_tutorial.html).  Also note that for simple data (e.g., one particle group), you can use the [ArrayReader](https://firefly.rcs.northwestern.edu/docs/reference/api/readers/firefly.data_reader.ArrayReader.html) class.

In [3]:
from firefly.data_reader import ParticleGroup, Reader

In [4]:
# create two Firefly particle groups
sphere = ParticleGroup(
    'sphere',
    coords_sphere,
    field_arrays=[fields_sphere], 
    field_names=['rands']
) 

cube = ParticleGroup(
    'cube',
    coords_cube,
    field_arrays=[fields_cube], 
    field_names=['randc']
) 

# create the Reader
my_reader = Reader()

# add the particle groups to the reader
my_reader.addParticleGroup(sphere)
my_reader.addParticleGroup(cube)


Make sure each field_array (1) has a field_filter_flag (0), assuming True.
Make sure each field_array (1) has a field_colormap_flag (0), assuming True.
Make sure each field_array (1) has a field_radius_flag (0), assuming False.
Make sure each field_array (1) has a field_filter_flag (0), assuming True.
Make sure each field_array (1) has a field_colormap_flag (0), assuming True.
Make sure each field_array (1) has a field_radius_flag (0), assuming False.
datadir is None, defaulting to /Users/daniel/opt/anaconda3/envs/firefly-env/lib/python3.11/site-packages/firefly/static/data/Data


## Display Inline

In [5]:
from firefly.server import spawnFireflyServer,quitAllFireflyServers

In [6]:
# define the local port (typically anything in 5000 - 8000 range)
port = 5500

In [7]:
process = spawnFireflyServer(port, method = 'flask')

Waiting up to 10 seconds for background Firefly server to start.....

 * Restarting with stat


.done! Your server is available at - http://localhost:5500


In [8]:
from IPython.display import IFrame
url = f'http://localhost:{port:d}/combined'
IFrame(url, width=1000, height=500)

In [9]:
# Send data to the server.
# Wait until it loads to run this command
my_reader.sendDataViaFlask()

sphere - 20000/20000 particles - 1 tracked fields
cube - 20000/20000 particles - 1 tracked fields
Posting data on port 5500...data posted!


## Get the selected data in Python

(after using the selection tool)

In [20]:
import requests

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# send a get request to receive the current settings from Firefly
# for larger amounts of data, you will need to increase the waitTime (in seconds) via params (see below; the default is 10s)
r = requests.get(url = f'http://localhost:{port:d}/get_selected_data', params = {'waitTime':60})
if r.status_code == 200:
    # success
    selection = r.json()
    # as a check
    partsKeys = list(selection.keys())
    print(partsKeys)
    print(selection[partsKeys[0]]['Coordinates_flat'][:100]) 
else:
    print('Error: {}'.format(r.status_code), r.content)


In [ ]:
# plot x, y for the selected points
partsKeys = list(selection.keys())
part0 = selection[partsKeys[0]]

# UPDATE THIS
x = part0['Coordinates_flat'][0::3]
y = part0['Coordinates_flat'][1::3]

f, ax = plt.subplots()
ax.scatter(x[:1000],y[:1000])

### Quit the Firefly server

... this doesn't always work in a notebook ... you can also quit the server by resetting the kernel.

In [ ]:
return_code = quitAllFireflyServers()